## オーケストレーターエージェント A2A のセットアップ

前のモジュールでは、A2A 呼び出しをサポートする AgentCore Runtime を使用して2つのエージェントを起動しました。

このラボでは、サブエージェントを呼び出すオーケストレーターを追加します。

<img src="images/architecture.png" style="width: 80%;">

それでは始めましょう！

### セットアップ

必要な依存関係をインポート

In [ ]:
# ライブラリをインポート
import os
import json
import requests
import boto3
from boto3.session import Session
from strands.tools import tool

# boto セッションを取得
boto_session = Session()
region = boto_session.region_name

前のラボからの情報を取得して、このラボで使用できるようにします。

In [ ]:
%store -r

### 1 - オーケストレーターエージェントのコードを作成

オーケストレーターに使用する Python コードを生成し、後で AgentCore にデプロイします。

In [ ]:
%%writefile agents/orchestrator.py
import logging
import json
import asyncio
from typing import Dict, Optional
from urllib.parse import quote
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

from helpers.utils import get_cognito_secret, reauthenticate_user, get_ssm_parameter, SSM_DOCS_AGENT_ARN, SSM_BLOGS_AGENT_ARN

from strands import Agent, tool
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from fastapi import HTTPException

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ハングを防ぐためにタイムアウトを短縮
DEFAULT_TIMEOUT = 15  # 300s から 15s に短縮
AGENT_TIMEOUT = 10    # エージェント呼び出しごとに 10s

# グローバルキャッシュとコネクションプール
_cache = {
    'cognito_config': None,
    'agent_arns': {},
    'agent_cards': {},
    'http_client': None
}

app = BedrockAgentCoreApp()

def get_cached_config():
    """高コストな操作をすべてキャッシュ"""
    if not _cache['agent_arns']:
        _cache['agent_arns'] = {
            'docs': get_ssm_parameter(SSM_DOCS_AGENT_ARN),
            'blogs': get_ssm_parameter(SSM_BLOGS_AGENT_ARN)
        }
    
    if not _cache['cognito_config']:
        secret = json.loads(get_cognito_secret())
        _cache['cognito_config'] = {
            'client_id': secret.get("client_id"),
            'client_secret': secret.get("client_secret")
        }
    
    return _cache['agent_arns'], _cache['cognito_config']

def get_bearer_token():
    """各リクエストに新しいベアラートークンを生成"""
    _, config = get_cached_config()
    return reauthenticate_user(
        config['client_id'], 
        config['client_secret']
    )

def get_http_client():
    """積極的なタイムアウトで HTTP クライアントを再利用"""
    if not _cache['http_client']:
        _cache['http_client'] = httpx.AsyncClient(
            timeout=httpx.Timeout(DEFAULT_TIMEOUT, connect=5.0),
            limits=httpx.Limits(max_keepalive_connections=5, max_connections=10),
            http2=True  # パフォーマンス向上のため HTTP/2 を有効化
        )
    return _cache['http_client']

def create_message(text: str) -> Message:
    return Message(
        kind="message",
        role=Role.user,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_agent_message(message: str, agent_type: str) -> Optional[str]:
    """サーキットブレーカーパターンを使用した最適化されたエージェント通信"""
    try:
        agent_arns, _ = get_cached_config()
        agent_arn = agent_arns[agent_type]
        bearer_token = get_bearer_token()
        
        from boto3.session import Session
        region = Session().region_name
        
        escaped_arn = quote(agent_arn, safe='')
        runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations/"
        
        headers = {
            "Authorization": f"Bearer {bearer_token}",
            'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': str(uuid4())
        }
        
        httpx_client = get_http_client()
        httpx_client.headers.update(headers)
        
        # エージェントカードをキャッシュ
        if agent_arn not in _cache['agent_cards']:
            resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
            _cache['agent_cards'][agent_arn] = await asyncio.wait_for(
                resolver.get_agent_card(), timeout=5.0
            )
        
        agent_card = _cache['agent_cards'][agent_arn]
        
        # 非ストリーミングモードでクライアントを作成
        config = ClientConfig(httpx_client=httpx_client, streaming=False)
        factory = ClientFactory(config)
        client = factory.create(agent_card)
        
        msg = create_message(message)
        
        # 操作全体にタイムアウトを設定
        async with asyncio.timeout(AGENT_TIMEOUT):
            async for event in client.send_message(msg):
                if isinstance(event, Message):
                    return event.parts[0].text if event.parts else "レスポンスなし"
                elif isinstance(event, tuple) and len(event) == 2:
                    return event[0].parts[0].text if event[0].parts else "レスポンスなし"
        
        return "タイムアウト: レスポンスを受信できませんでした"
        
    except asyncio.TimeoutError:
        logger.warning(f"{agent_type} エージェント呼び出しがタイムアウト")
        return f"エージェント {agent_type} がタイムアウトしました"
    except Exception as e:
        logger.error(f"{agent_type} 呼び出しエラー: {e}")
        return f"エラー: {str(e)[:100]}"

@tool
async def send_mcp_message(message: str):
    """タイムアウト付きで AWS Docs エージェントにメッセージを送信"""
    return await send_agent_message(f"Summarize briefly: {message}", 'docs')

@tool
async def send_blog_message(message: str):
    """タイムアウト付きで AWS Blogs エージェントにメッセージを送信"""
    return await send_agent_message(f"Summarize briefly: {message}", 'blogs')


system_prompt = """あなたは AWS 情報オーケストレーターです。

利用可能なエージェント:
- AWS Documentation: AWS サービスの技術的詳細
- AWS Blogs: 最新の AWS ニュースと発表

重要: レスポンスは短く、高速に。常にサブエージェントに要約をリクエストしてください。

ガイドライン:
- 可能な場合は並列クエリを使用
- エージェントごとに10秒後にタイムアウト
- 迅速で実用的な回答を提供
- エージェントがタイムアウトした場合は、知っている情報を提供
"""

agent = Agent(
    system_prompt=system_prompt, 
    tools=[send_mcp_message, send_blog_message],
    name="AWS Orchestration Agent",
    description="An agent to orchestrate sub-agents"
)

@app.entrypoint
async def invoke_agent(payload, context):
    logger.info("高速オーケストレーターがリクエストを処理中")
    
    try:
        user_prompt = payload.get("prompt", "")
        if not user_prompt:
            raise HTTPException(status_code=400, detail="プロンプトが提供されていません")

        logger.info(f"クエリ: {user_prompt[:100]}...")
        
        # 操作全体に全体タイムアウトを設定
        async with asyncio.timeout(25.0):  # 最大 25s
            agent_stream = agent.stream_async(user_prompt)
            
            async for event in agent_stream:
                yield event

    except asyncio.TimeoutError:
        logger.error("操作全体がタイムアウトしました")
        yield {"error": "リクエストが 25 秒後にタイムアウトしました"}
    except HTTPException:
        raise
    except Exception as e:
        logger.error(f"処理失敗: {e}")
        yield {"error": f"処理失敗: {str(e)[:100]}"}

# シャットダウン時のクリーンアップ
async def cleanup():
    if _cache['http_client']:
        await _cache['http_client'].aclose()

if __name__ == "__main__":
    import atexit
    atexit.register(lambda: asyncio.run(cleanup()))
    app.run()

#### 1.1 - エージェント用の IAM ロールを作成

In [ ]:
from helpers.utils import create_agentcore_runtime_execution_role, ORCHESTRATOR_ROLE_NAME

agent_name="aws_orchestrator_assistant"

execution_role_arn = create_agentcore_runtime_execution_role(ORCHESTRATOR_ROLE_NAME)

### 2 - AgentCore Runtime へのデプロイ

それでは、オーケストレーターを AgentCore Runtime にデプロイしましょう。

この例では、`protocol` パラメータを追加していないことに注意してください。これは、HTTP エージェントになることを意味します。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime = Runtime()

# デプロイメントを設定
response = agentcore_runtime.configure(
    entrypoint="agents/orchestrator.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [COGNITO_CLIENT_ID],
            "discoveryUrl": DISCOVERY_URL,
        }
    },
)

print("設定完了:", response)

In [ ]:
launch_result = agentcore_runtime.launch()
print("起動完了:", launch_result.agent_arn)

agent_arn = launch_result.agent_arn

**デプロイステータスの確認**

デプロイが完了したか確認しましょう：

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

print(f"最終ステータス: {status}")

#### 2.1 - 出力のエクスポートと保存

クリーンアップノートブックで使用する変数をエクスポート：

In [ ]:
ORCHESTRATION_ID = launch_result.agent_id
ORCHESTRATION_ARN = launch_result.agent_arn
ORCHESTRATION_NAME = agent_name

%store ORCHESTRATION_ID
%store ORCHESTRATION_ARN
%store ORCHESTRATION_NAME

### 3 - オーケストレーターエージェントを使用した A2A エージェントの呼び出し

まず、認証トークンを更新しましょう：

In [ ]:
from helpers.utils import reauthenticate_user

bearer_token = reauthenticate_user(
    COGNITO_CLIENT_ID,
    COGNITO_SECRET
)

それでは、A2A を使用して最初のエージェントを呼び出し、AWS Docs を確認するためにオーケストレーターを呼び出してみましょう：

In [ ]:
import requests
import json
import uuid
from urllib.parse import quote

session_id = str(uuid.uuid4())
print(f'セッション用に呼び出し中: {session_id}')

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
}

prompt = {"prompt": "What is DynamoDB?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

In [ ]:
import uuid

session_id = str(uuid.uuid4())
print(f'セッション用に呼び出し中: {session_id}')

headers = {
    'Authorization': f'Bearer {bearer_token}',
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id
}

prompt = {"prompt": "Give me the latest published blog for Bedrock AgentCore?"}

escaped_agent_arn = quote(ORCHESTRATION_ARN, safe='')

response = requests.post(
    f'https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations',
    headers=headers,
    data=json.dumps(prompt)
)

for line in response.iter_lines(decode_unicode=True):
    if line.startswith('data: '):
        data = line[6:]
        try:
            parsed = json.loads(data)
            print(parsed)
        except:
            print(data)

おめでとうございます！Amazon AgentCore Runtime で A2A プロトコルを使用した完全なソリューションをデプロイしました。